In [43]:
# Import lib
# ===========================================================
import csv
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SMOTENC
import random
import time
import collections
import math
import sys
from tqdm import tqdm
from time import sleep
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

from datascience import *
from scipy import stats

import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

In [3]:
# Initialize useful data
# ===========================================================
# with open('clinvar_conflicting_mapped.csv', 'r') as f:
#     reader = csv.reader(f)
#     temp_rows = list(reader)
df = pd.read_csv('clinvar_conflicting_mapped.csv', low_memory=False)
columns_backup = df.columns
attributes = list(df.columns)
attribute_dimension = len(attributes) - 1 # eliminate the CLASS column
all_rows = df.values.tolist()
row_num = len(all_rows)
df = df.sample(n = df.shape[0])
df.head()

,CHROM,POS,REF,ALT,AF_ESP,AF_EXAC,AF_TGP,CLNDISDB,CLNDN,CLNHGVS,...,Codons,STRAND,BAM_EDIT,SIFT,PolyPhen,LoFtool,CADD_PHRED,CADD_RAW,BLOSUM62,CLASS
5555,0.000000,0.953783,0.764434,0.567686,0.000000,0.550472,0.000000,0.774746,0.422138,0.606124,...,0.416029,0.0625,0.333333,0.6,0.2,0.122966,0.727151,0.550320,0.000404,0
54326,0.625000,0.942312,0.815242,0.661572,0.000000,0.000000,0.000000,0.835608,0.819546,0.277935,...,0.745610,0.9375,0.666667,0.2,0.2,0.966901,0.723886,0.410247,0.000000,0
25818,0.083333,0.181241,0.578522,0.746725,0.793807,0.953202,0.563967,0.366905,0.686501,0.973109,...,0.188654,0.9375,0.666667,0.8,0.8,0.374075,0.723886,0.527021,0.000000,0
15239,0.666667,0.181177,0.314088,0.661572,0.000000,0.923504,0.515573,0.085229,0.956156,0.572866,...,0.671769,0.9375,0.333333,0.8,0.8,0.586908,0.731951,0.723877,0.000000,0
14503,0.666667,0.846788,0.578522,0.746725,0.000000,0.772911,0.000000,0.115118,0.726242,0.245919,...,0.237731,0.9375,0.333333,0.0,0.0,0.758136,0.003168,0.544248,0.999924,0


In [11]:
# SMOTE Sampling for imbalanced input data
# ===========================================================
X_train = [row[: -1] for row in all_rows]
y_train = [row[-1] for row in all_rows]
# smt = SMOTENC(random_state=42, categorical_features=cate_columns)
# X_train, y_train = smt.fit_resample(X_train, y_train)
smt = SMOTE()
X_train, y_train = smt.fit_sample(X_train, y_train)

In [12]:
# Overwrite the original all_rows with the re-sampled data
# ===========================================================
all_rows = np.zeros((X_train.shape[0], X_train.shape[1] + 1))
all_rows[:, :X_train.shape[1]] = X_train
all_rows[:, X_train.shape[1]] = y_train
df = pd.DataFrame(all_rows)
df.columns = columns_backup
df = df.sample(n = df.shape[0])
df = df.sample(n = df.shape[0])
df.astype({'CLASS': 'int32'})

all_rows = df.values.tolist()
row_num = len(all_rows)
df.head()
# np.bincount([row[-1] for row in all_rows])

,CHROM,POS,REF,ALT,AF_ESP,AF_EXAC,AF_TGP,CLNDISDB,CLNDN,CLNHGVS,...,Codons,STRAND,BAM_EDIT,SIFT,PolyPhen,LoFtool,CADD_PHRED,CADD_RAW,BLOSUM62,CLASS
22198,0.541667,0.251715,0.815242,0.661572,0.339198,0.948703,0.196933,0.665259,0.069870,0.950313,...,0.607834,0.9375,0.666667,0.8,0.8,0.973558,0.437404,0.561492,0.000000,0.0
8319,0.000000,0.324281,0.578522,0.746725,0.457776,0.454177,0.171059,0.811458,0.101080,0.218951,...,0.325529,0.9375,0.666667,0.8,0.8,0.498151,0.659466,0.181909,0.000000,0.0
26274,0.083333,0.027490,0.684758,0.799127,0.000000,0.426279,0.126018,0.981698,0.157127,0.904047,...,0.312472,0.9375,0.333333,0.6,0.2,0.480954,0.673579,0.647353,0.000025,1.0
90527,0.041667,0.376384,0.760738,0.664360,0.188264,0.905534,0.000000,0.647627,0.842729,0.755409,...,0.350778,0.9375,0.333333,0.6,0.6,0.266457,0.744822,0.620799,0.000015,1.0
33117,0.708333,0.522253,0.815242,0.661572,0.000000,0.978851,0.217537,0.439138,0.579806,0.203979,...,0.671769,0.9375,0.333333,0.8,0.8,0.193787,0.593222,0.134787,0.000000,1.0


In [13]:
# Divide whole dataset into training set and testing set
# ===========================================================
training_percentage = 0.5  # percent of partition of training dataset
training_size = int(row_num * training_percentage)
testing_size = row_num - training_size
training_data = all_rows[: training_size]  # training data should include header row
testing_data = all_rows[training_size: ]   # testing data don't need to include header row
# np.bincount([row[-1] for row in training_data])
# np.bincount([row[-1] for row in testing_data])

# Instruction
1. Select the number of clusters you want to identify in your data 
2. Randomly select k distinct data points 
3. Measure the distance between the 1st point and the k initial clusters 
4. Assign the 1st point to the nearest cluster 
5. Iterate through all points and do step 3 & 4 
6. Calculate the mean of each cluster 
7. Use the calculated mean of each cluster as k new initial data points and restart from 3 
8. Loop until the mean converge 
9. Do Step 1 - 8 for n times, select the best one 

In [27]:

# i have rescaled all data to confirm that each input dimension ranges within [0, 1]

def gen_random_pivots(k, dim):
    return [np.random.rand(1, dim) for _ in range(k)]

def euclidean_distance(pivot, point):
    # point can have a CLASS entry, we don't want to use it, so eliminate it outside of the function
    
    return np.linalg.norm(np.subtract(pivot, point))

def find_nearest_pivot(point):
    winner = np.random.rand(1, attribute_dimension)
    min_dist = float('inf')
    for i in range(len(pivots)):
        pivot = pivots[i][0]
        temp_dist = euclidean_distance(pivot, point[: -1])
        idx = -2
        if temp_dist < min_dist:
            winner, min_dist, idx = pivot, temp_dist, i
    return winner, idx

def cluster_mean_point(cluster):
    mean_point = np.zeros((1, attribute_dimension))
    for point in cluster:
        mean_point = np.add(mean_point, point[: -1])
    mean_point = np.divide(mean_point, len(cluster) + 0.00000001)
    return mean_point

def clusters_acc(clusters):
    counter1, counter2 = collections.Counter([point[-1] for point in clusters[0]]), collections.Counter([point[-1] for point in clusters[1]])
    
    one_rate1 = counter1.get(1, 0) / (counter1.get(1, 0) + counter1.get(0, 0) + 0.00000001)
    zero_rate1 = counter1.get(0, 0) / (counter1.get(1, 0) + counter1.get(0, 0) + 0.00000001)
    one_rate2 = counter2.get(1, 0) / (counter2.get(1, 0) + counter2.get(0, 0) + 0.00000001)
    zero_rate2 = counter2.get(0, 0) / (counter2.get(1, 0) + counter2.get(0, 0) + 0.00000001)
    dataset_size = sum(counter1.values()) + sum(counter2.values())
    if one_rate1 > one_rate2:
        # counter1 -> label 1, counter2 -> label 0
        return (counter1.get(1, 0) + counter2.get(0, 0)) / dataset_size
    else:
        return (counter1.get(0, 0) + counter2.get(1, 0)) / dataset_size
    '''
    return (counter1.get(0, 0) + counter2.get(1, 0)) / (sum(counter1.values()) + sum(counter2.values()))
    '''

In [28]:
# unsupervised clustering
# ==============================================
K = 2
n = 15 # number of try to run
converge_radius = 0.002

print("Training_size: %d" % training_size)
winner = []
min_var = float('inf')
max_acc = -float('inf')

for i in range(n):
    
    # randomly select K distinct data points
    pivots = gen_random_pivots(K, attribute_dimension)
    
    sys.stdout.write('\r')
    # the exact output you're looking for:
    sys.stdout.write("Training: [%-20s] %d%%" % ('='*int((i + 1) * (100 / n) / 5), int((i + 1) * (100 / n))))
    sys.stdout.flush()
#     sleep(0.25)
    
    while True:
        # clusters = dict() # i think it would be better to instantize this with a dictionary
        clusters = [[] for _ in range(K)] # init K empty clusters
    
        for point in training_data:
            
            # each row of training_data is a high-dimensional point
            _, idx_of_pivot = find_nearest_pivot(point)
            clusters[idx_of_pivot].append(point)
        temp_pivots = [cluster_mean_point(cluster) for cluster in clusters]
        
        # loop until init_pivots converge
        if np.linalg.norm(np.subtract(pivots, temp_pivots)) < converge_radius:
            break
        pivots = list(temp_pivots)
    
    # pick the best clustering with the most difference between each other
    cluster_sizes = [len(cluster) for cluster in clusters]
    temp_var = np.var(cluster_sizes)
    temp_acc = clusters_acc(clusters)
    if temp_acc > max_acc:
        winner = list(clusters)
        max_acc = temp_acc

'''
* TP: Prediction is True + Predicted value is Positive
* FP: Prediction is False + Predicted value is Positive
* TN: Prediction is True + Predicted value is Negative
* FN: Prediction is False + Predicted value is Negative

* Accuracy = $\frac{TP + TN}{TP + FN + FP + TN}$
* Sensitivity (TPR) = $\frac{TP}{TP + FN}$
* Specificity (FPR) = $\frac{TN}{TN + FP}$
'''

# cause we don't really have cutoff points here, so computing TP, .. would be meaningless
# here we only care about Acc
print("\nwinner: ", len(winner[0]), len(winner[1]))
print("Training Acc: ", max_acc)
counter1, counter2 = collections.Counter([point[-1] for point in winner[0]]), collections.Counter([point[-1] for point in winner[1]])
print(counter1, counter2)

training_size: 48754
Training: [====================] 100%
winner:  27604 21150
training Acc:  0.5164089100381507
Counter({0.0: 14208, 1.0: 13396}) Counter({1.0: 10969, 0.0: 10181})


In [36]:
# testing
# ===================================================================
# pivots
test_clusters = [[] for _ in range(K)]
cutoff = 0.5
for point in testing_data:
    if euclidean_distance(pivots[0], point[: -1]) / euclidean_distance(pivots[1], point[: -1]) < cutoff / (1 - cutoff):
        test_clusters[0].append(point)
    else:
        test_clusters[1].append(point)
print("testing Acc", clusters_acc(test_clusters))
print(len(test_clusters[0]), len(test_clusters[1]))

testing Acc 0.5111580588259425
20946 27808


In [39]:
# Compute TN, TP, FN, FP, etc.
# ===========================================================
ROC = Table(make_array('CUTOFF', 'TN', 'FN', 'FP', 'TP', 'ACC'))
step_size = 0.05
for cutoff in np.arange(0, 1 + step_size, step_size):
    
    sys.stdout.write('\r')
    # the exact output you're looking for:
    sys.stdout.write("Testing: [%-20s] %d%%" % ('='*int(cutoff * 100 / 5), int(cutoff * 100)))
    sys.stdout.flush()
    
    test_clusters = clusters = [[] for _ in range(K)]
    for point in testing_data:
        if euclidean_distance(pivots[0], point[: -1]) / euclidean_distance(pivots[1], point[: -1]) < cutoff / (1 - cutoff):
            test_clusters[0].append(point)
        else:
            test_clusters[1].append(point)
    counter1, counter2 = collections.Counter([point[-1] for point in test_clusters[0]]), collections.Counter([point[-1] for point in test_clusters[1]])
    one_rate1 = counter1.get(1, 0) / (counter1.get(1, 0) + counter1.get(0, 0) + 0.00000001)
    zero_rate1 = counter1.get(0, 0) / (counter1.get(1, 0) + counter1.get(0, 0) + 0.00000001)
    one_rate2 = counter2.get(1, 0) / (counter2.get(1, 0) + counter2.get(0, 0) + 0.00000001)
    zero_rate2 = counter2.get(0, 0) / (counter2.get(1, 0) + counter2.get(0, 0) + 0.00000001)
    dataset_size = sum(counter1.values()) + sum(counter2.values())
    if one_rate1 > one_rate2:
        # counter1 -> label 1, counter2 -> label 0
        TP = counter1.get(1, 0) / (sum(counter1.values()) + 0.00000001)
        FN = counter1.get(0, 0) / (sum(counter1.values()) + 0.00000001)
        FP = counter2.get(1, 0) / (sum(counter2.values()) + 0.00000001)
        TN = counter2.get(0, 0) / (sum(counter2.values()) + 0.00000001)
    else:
        # counter1 -> label 0, counter2 -> label 1
        TP = counter1.get(1, 0) / (sum(counter1.values()) + 0.00000001)
        FN = counter1.get(0, 0) / (sum(counter1.values()) + 0.00000001)
        FP = counter2.get(1, 0) / (sum(counter2.values()) + 0.00000001)
        TN = counter2.get(0, 0) / (sum(counter2.values()) + 0.00000001)
    output = [cutoff, TN, FN, FP, TP]
    acc = (output[1] + output[4]) / (sum(output[1:]) + 0.00000001)
    output.append(acc)
    ROC = ROC.with_row(output)
ROC = ROC.with_columns('SENSITIVITY', ROC.apply(lambda TP, FN: TP / (TP + FN + 0.00000001), 'TP', 'FN'))
ROC = ROC.with_columns('FPR', ROC.apply(lambda TN, FP: FP / (TN + FP + 0.00000001), 'TN', 'FP'))
ROC = ROC.with_column('FMEAS', ROC.apply(lambda TP, FP, FN: 2 * (TP / (TP + FN)) * (TP / (TP + FP)) / (TP / (TP + FN) + TP / (TP + FP)), 'TP', 'FP', 'FN'))

Testing: [====================] 1400%

/Users/frostace/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in double_scalars
  
/Users/frostace/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in double_scalars


In [40]:
ROC.show()

CUTOFF,TN,FN,FP,TP,ACC,SENSITIVITY,FPR,FMEAS
0,0.499754,0,0.500246,0,0.499754,0,0.500246,nan
0.05,0.499754,0,0.500246,0,0.499754,0,0.500246,nan
0.1,0.499754,0,0.500246,0,0.499754,0,0.500246,nan
0.15,0.499754,0,0.500246,0,0.499754,0,0.500246,nan
0.2,0.499754,0,0.500246,0,0.499754,0,0.500246,nan
0.25,0.499754,0,0.500246,0,0.499754,0,0.500246,nan
0.3,0.499754,0,0.500246,0,0.499754,0,0.500246,nan
0.35,0.499754,0,0.500246,0,0.499754,0,0.500246,nan
0.4,0.500021,0.275862,0.499979,0.724138,0.612079,0.724138,0.499979,0.651169
0.45,0.502423,0.48676,0.497577,0.51324,0.507832,0.51324,0.497577,0.510479


In [45]:
# Acc Curve by cutoff
# ===========================================================
matplotlib.use('TkAgg')
fig = plt.figure()
plt.xlabel('Cutoff')
plt.ylabel('Accuracy')
plt.title('Accuracy - Cutoff of K means')
plt.plot(ROC.column('CUTOFF'), ROC.column('ACC'), color='orange')
plt.axis([0, 1, 0, 1])
plt.show()
fig.savefig('K means ACC.png', bbox_inches='tight')

In [46]:
# ROC_CURVE
# ===========================================================
fig = plt.figure()
plt.xlabel('False Positive Rate')
plt.ylabel('Sensitivity')
plt.title('ROC - Curve of K means')
plt.plot(ROC.column('FPR'), ROC.column('SENSITIVITY'), color='orange')
plt.plot(np.arange(0, 1.1, 0.1), np.arange(0, 1.1, 0.1), color='black')
plt.legend(['K means', 'Null'])
plt.axis([0, 1, 0, 1])
plt.show()
fig.savefig('K means ROC.png', bbox_inches='tight')

In [ ]:
# Compute AUC
# ===========================================================
length = len(ROC.column('FPR'))
auc = 0
for i in range(length - 1):
    auc += 0.5 * abs(ROC.column('FPR')[i + 1] - ROC.column('FPR')[i]) * (ROC.column('SENSITIVITY')[i] + ROC.column('SENSITIVITY')[i + 1])
print("auc = %.03f" %auc)

In [20]:
len(test_clusters[0])
len(test_clusters[1])

97508

In [21]:
len(testing_data)

48754

In [34]:
shit = [[1,2,3], [2,3,4], [5,6,7]]
fuck = pd.DataFrame(shit)

In [48]:
fuck.sample(n=3)

,0,1,2
1,2,3,4
2,5,6,7
0,1,2,3


In [57]:
print(np.bincount([row[-1] for row in training_data]))
print(np.bincount([row[-1] for row in testing_data]))

[24245 24509]
[24509 24245]


In [26]:
pivots[0][0]

array([5.05485953e-01, 4.98600783e-01, 7.05735926e-01, 6.93868864e-01,
       1.55702479e-01, 3.45387305e-01, 1.51277261e-01, 5.08254154e-01,
       5.25895191e-01, 5.01116503e-01, 1.43479569e-01, 4.21078110e-01,
       3.60917283e-01, 1.69799535e-01, 6.91144906e-01, 8.24555661e-01,
       7.48878883e-01, 4.68971079e-01, 0.00000000e+00, 4.86097082e-01,
       6.66666667e-01, 4.02138345e-01, 4.34053455e-06, 5.07474431e-01,
       5.09164776e-01, 5.11010470e-01, 5.00192490e-01, 4.88280567e-01,
       5.05545288e-01, 4.81289729e-01, 3.52809754e-01, 3.46486059e-01,
       5.02569839e-01, 4.00908505e-01, 4.99113437e-01, 5.06418546e-01])